<a href="https://colab.research.google.com/github/jsleocadio/IMD0186-MINERACAO_DE_DADOS/blob/main/Atividade_de_%C3%8Dndices_de_valida%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Execute os seguintes passos:

## 1. Baixe a base de dados Image.csv (https://www.dropbox.com/sh/f6i1fj8qpjpud9c/AABMYBYfc7jTOZHB_qHPdGBa?dl=0 ). Chame esta base de Base-Original.

In [1]:
!wget -q -O "file.zip" "https://www.dropbox.com/scl/fo/g23norkdqy1z0y124zklf/ABUWZBocbv5tc3zLt1K14ak?rlkey=8swc156wdq9aqr11teh3dou0w&e=1&st=gafhq3s1&dl=1"
!unzip -q -p file.zip Image.csv > Image.csv

In [4]:
import pandas as pd

df = pd.read_csv('Image.csv')
labels = df['Class']
BaseOriginal = df.drop(columns=['Class'])
BaseOriginal.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,att17,att18,att19
0,140,125,9,0.0,0.0,0.277778,0.062963,0.666667,0.311111,6.185185,7.333334,7.666666,3.555556,3.444444,4.444445,-7.888889,7.777778,0.545635,-1.121818
1,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
2,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
3,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
4,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773


## 2. Crie duas versões dessa base, aplicando os métodos de redução:

In [6]:
def train_knn_and_get_accuracy(df, labels):
  from sklearn.model_selection import train_test_split
  from sklearn.neighbors import KNeighborsClassifier
  from sklearn.metrics import accuracy_score

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2, random_state=42)
  # Initialize KNN classifier
  knn = KNeighborsClassifier()
  # Fit the model
  knn.fit(X_train, y_train)
  # Make predictions
  y_pred = knn.predict(X_test)
  # Calculate accuracy
  accuracy = accuracy_score(y_test, y_pred)
  return accuracy

In [9]:
def get_best_base(base1, base2, labels):
  accuracy1 = train_knn_and_get_accuracy(base1, labels)
  accuracy2 = train_knn_and_get_accuracy(base2, labels)
  print(f"Acurácia da base 1: {accuracy1}")
  print(f"Acurácia da base 2: {accuracy2}")
  if accuracy1 > accuracy2:
    return base1
  else:
    return base2

### a. Correlação (escolha a melhor versão das duas bases que vc criou);

In [5]:
def get_correlation_df(df, nsmallest):
  import numpy as np

  # Get the correlation matrix
  correlation_matrix = df.corr()
  # Set diagonal values to NaN to exclude self-correlations
  np.fill_diagonal(correlation_matrix.values, np.nan)
  # Unstack the correlation matrix and sort by correlation values
  sorted_correlations = correlation_matrix.unstack().sort_values(ascending=False)
  # Filter for correlations greater than 0
  positive_correlations = sorted_correlations[sorted_correlations != 0]
  # Get the n smallest positive correlations
  smallest_positive_correlations = positive_correlations.nsmallest(nsmallest)
  # Get the columns corresponding to the smallest positive correlations
  columns_base = []
  for i in range(len(smallest_positive_correlations.index)):
    columns_base.append(smallest_positive_correlations.index[i][0])
  return df[columns_base]

In [7]:
BaseReduzidaCorr1 = get_correlation_df(BaseOriginal, 3)
BaseReduzidaCorr1.head()

,att17,att14,att12
0,7.777778,3.444444,7.666666
1,8.444445,5.000000,7.777778
2,7.555555,4.333334,7.222222
3,7.777778,5.777778,6.444445
4,7.888889,2.888889,7.666666


In [8]:
BaseReduzidaCorr2 = get_correlation_df(BaseOriginal, 9)
BaseReduzidaCorr2.head()

,att17,att14,att12,att14,att14,att15,att15,att16,att10
0,7.777778,3.444444,7.666666,3.444444,3.444444,4.444445,4.444445,-7.888889,6.185185
1,8.444445,5.000000,7.777778,5.000000,5.000000,3.333333,3.333333,-8.333333,6.666666
2,7.555555,4.333334,7.222222,4.333334,4.333334,3.333333,3.333333,-7.666666,6.111111
3,7.777778,5.777778,6.444445,5.777778,5.777778,1.777778,1.777778,-7.555555,5.851852
4,7.888889,2.888889,7.666666,2.888889,2.888889,4.888889,4.888889,-7.777778,6.037037


In [10]:
BaseReduzidaCorr = get_best_base(BaseReduzidaCorr1, BaseReduzidaCorr2, labels)
BaseReduzidaCorr.head()

Acurácia da base 1: 0.8614718614718615
Acurácia da base 2: 0.8961038961038961


,att17,att14,att12,att14,att14,att15,att15,att16,att10
0,7.777778,3.444444,7.666666,3.444444,3.444444,4.444445,4.444445,-7.888889,6.185185
1,8.444445,5.000000,7.777778,5.000000,5.000000,3.333333,3.333333,-8.333333,6.666666
2,7.555555,4.333334,7.222222,4.333334,4.333334,3.333333,3.333333,-7.666666,6.111111
3,7.777778,5.777778,6.444445,5.777778,5.777778,1.777778,1.777778,-7.555555,5.851852
4,7.888889,2.888889,7.666666,2.888889,2.888889,4.888889,4.888889,-7.777778,6.037037


### b. PCA (escolha a melhor versão das duas bases que vc criou).

In [11]:
def get_pca_df(df, n_components):
  import pandas as pd
  from sklearn.decomposition import PCA

  pca = PCA(n_components=n_components)
  return pd.DataFrame(pca.fit_transform(df))

In [13]:
BaseReduzidaPCA1 = get_pca_df(BaseOriginal, 0.9)
BaseReduzidaPCA1.head()

,0,1,2,3
0,-69.100415,-18.168883,-7.232970,29.610680
1,-67.513749,-66.637864,-8.779705,26.020605
2,-77.876074,15.025764,-6.545274,14.068556
3,-83.734129,85.681604,-4.718357,10.204779
4,-72.106699,83.585227,-4.047370,33.969377


In [14]:
BaseReduzidaPCA2 = get_pca_df(BaseOriginal, 3)
BaseReduzidaPCA2.head()

,0,1,2
0,-69.100415,-18.168883,-7.232970
1,-67.513749,-66.637864,-8.779705
2,-77.876074,15.025764,-6.545274
3,-83.734129,85.681604,-4.718357
4,-72.106699,83.585227,-4.047370


In [15]:
BaseReduzidaPCA = get_best_base(BaseReduzidaPCA1, BaseReduzidaPCA2, labels)
BaseReduzidaPCA.head()

Acurácia da base 1: 0.9090909090909091
Acurácia da base 2: 0.6753246753246753


,0,1,2,3
0,-69.100415,-18.168883,-7.232970,29.610680
1,-67.513749,-66.637864,-8.779705,26.020605
2,-77.876074,15.025764,-6.545274,14.068556
3,-83.734129,85.681604,-4.718357,10.204779
4,-72.106699,83.585227,-4.047370,33.969377


## 3. Para as 3 bases, execute o k-Means variando k de 2 até 20 utilizando ambas as bases transformadas, e através do DB, Silhouette e CR avalie as partições geradas.


In [16]:
def get_indices_kmeans(df, labels, n_clusters):
  from sklearn.metrics import davies_bouldin_score, silhouette_score, adjusted_rand_score
  from sklearn.cluster import KMeans

  km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=300, n_init=10, random_state=37)
  km.fit(df)
  km_labels = km.fit_predict(df)
  db = davies_bouldin_score(df, km_labels)
  sil = silhouette_score(df, km_labels, metric='euclidean')
  cr = adjusted_rand_score(labels, km_labels)
  return db, sil, cr

In [20]:
def print_indices_kmeans(df1, df2, df3, labels):
   for i in range(2, 21):
    db1, sil1, cr1 = get_indices_kmeans(df1, labels, i)
    db2, sil2, cr2 = get_indices_kmeans(df2, labels, i)
    db3, sil3, cr3 = get_indices_kmeans(df3, labels, i)
    print(f"DB_k-means (k = {i}): Original: {db1}, Correlation: {db2}, PCA: {db3}")
    print(f"Sil_k-means (k = {i}): Original: {sil1}, Correlation: {sil2}, PCA: {sil3}")
    print(f"CR_k-means (k = {i}): Original: {cr1}, Correlation: {cr2}, PCA: {cr3}")

In [21]:
print_indices_kmeans(BaseOriginal, BaseReduzidaCorr, BaseReduzidaPCA, labels)

DB_k-means (k = 2): Original: 0.8293494920142007, Correlation: 0.6053505024509765, PCA: 0.797384519363615
Sil_k-means (k = 2): Original: 0.45763311150890745, Correlation: 0.621075180711827, PCA: 0.46408206139494074
CR_k-means (k = 2): Original: 0.10248312873427927, Correlation: 0.11666287789746198, PCA: 0.10254028089641205
DB_k-means (k = 3): Original: 0.7603654420792049, Correlation: 0.5047306782579821, PCA: 0.6889037877543901
Sil_k-means (k = 3): Original: 0.470645859004893, Correlation: 0.6089242893752276, PCA: 0.47943294334876463
CR_k-means (k = 3): Original: 0.10289888615600172, Correlation: 0.2973633086869038, PCA: 0.10289888615600172
DB_k-means (k = 4): Original: 0.9451806600023536, Correlation: 0.6394258188154598, PCA: 0.8780501975871191
Sil_k-means (k = 4): Original: 0.3636633155006026, Correlation: 0.5301749806148033, PCA: 0.3756927725819002
CR_k-means (k = 4): Original: 0.16234055893056248, Correlation: 0.3148874669623218, PCA: 0.16244462303481136
DB_k-means (k = 5): Origina

## 4. Gere um gráfico de linhas para que o método do Elbow possa ser visualizado.


In [25]:
def plot_line_kmeans(df1, df2, df3, labels, debug = False):
  import pandas as pd
  import plotly.graph_objects as go
  from plotly.subplots import make_subplots

  dados_db = []
  dados_sil = []
  dados_cr = []
  for i in range(2, 21):
    db1, sil1, cr1 = get_indices_kmeans(df1, labels, i)
    db2, sil2, cr2 = get_indices_kmeans(df2, labels, i)
    db3, sil3, cr3 = get_indices_kmeans(df3, labels, i)
    if debug:
      print(f"DB_k-means (k = {i}): Original: {db1}, Correlation: {db2}, PCA: {db3}")
      print(f"Sil_k-means (k = {i}): Original: {sil1}, Correlation: {sil2}, PCA: {sil3}")
      print(f"CR_k-means (k = {i}): Original: {cr1}, Correlation: {cr2}, PCA: {cr3}")
    dado_db = {'grupos': i, 'Original': db1, 'Correlation': db2, 'PCA': db3}
    dado_sil = {'grupos': i, 'Original': sil1, 'Correlation': sil2, 'PCA': sil3}
    dado_cr = {'grupos': i, 'Original': cr1, 'Correlation': cr2, 'PCA': cr3}
    dados_db.append(dado_db)
    dados_sil.append(dado_sil)
    dados_cr.append(dado_cr)

  df_db = pd.DataFrame(dados_db)
  df_sil = pd.DataFrame(dados_sil)
  df_cr = pd.DataFrame(dados_cr)

  fig = make_subplots(rows=1, cols=3, subplot_titles=('Davies-Bouldin Score', 'Silhouette Score', 'Adjusted Rand Score'))
  fig.add_trace(go.Scatter(x=df_db['grupos'], y=df_db['Original'], name='Original'), row=1, col=1)
  fig.add_trace(go.Scatter(x=df_db['grupos'], y=df_db['Correlation'], name='Correlation'), row=1, col=1)
  fig.add_trace(go.Scatter(x=df_db['grupos'], y=df_db['PCA'], name='PCA'), row=1, col=1)

  fig.add_trace(go.Scatter(x=df_sil['grupos'], y=df_sil['Original'], name='Original'), row=1, col=2)
  fig.add_trace(go.Scatter(x=df_sil['grupos'], y=df_sil['Correlation'], name='Correlation'), row=1, col=2)
  fig.add_trace(go.Scatter(x=df_sil['grupos'], y=df_sil['PCA'], name='PCA'), row=1, col=2)

  fig.add_trace(go.Scatter(x=df_cr['grupos'], y=df_cr['Original'], name='Original'), row=1, col=3)
  fig.add_trace(go.Scatter(x=df_cr['grupos'], y=df_cr['Correlation'], name='Correlation'), row=1, col=3)
  fig.add_trace(go.Scatter(x=df_cr['grupos'], y=df_cr['PCA'], name='PCA'), row=1, col=3)

  fig.update_layout(title='Gráficos de comparação', showlegend=True)
  fig.show()

In [26]:
plot_line_kmeans(BaseOriginal, BaseReduzidaCorr, BaseReduzidaPCA, labels)

# Para as três bases (Base-Original, Base-Reduzida-Corr e Base-Reduzida-PCA), analise os gráficos e responda as perguntas:


## a. Qual foi o melhor número de grupos, de acordo com o DB e Silhouette?

* Para o índice DB, a base com Correlação apresentou consistentemente os menores valores, especialmente para k = 3 (0.504) e k = 5 (0.597).

* Para o índice Silhouette, a base com Correlação também apresentou consistentemente os maiores valores, especialmente para k = 2 (0.621), k = 3 (0.609) e k = 5 (0.544).

Portanto, com base nos índices Davies-Bouldin e Silhouette, o melhor número de grupos (k) para a base reduzida por Correlação é:

* k = 3: Menor valor de DB (0.504) e segundo maior valor de Silhouette (0.609).
* k = 5: Baixo valor de DB (0.597) e maior valor de Silhouette (0.544).

Ambas as escolhas são válidas, mas se precisarmos escolher uma, podemos optar por k = 3 para balancear um ótimo índice DB e um excelente índice Silhouette.

## b. A melhor partição teve um grau de concordância alto com a partição original (classes)?

A maior concordância foi observada para 𝑘=5 na base reduzida por correlação (CR = 0.4440227166868555). Portanto, a partição gerada com 𝑘=5 na base reduzida por correlação teve o maior grau de concordância com a partição original (classes).

## c. O comportamento foi o mesmo tanto para a Correlação quanto para o PCA?

Para a maioria dos valores de 𝑘, a redução por correlação apresentou um maior grau de concordância (ARI) com a partição original em comparação com a redução por PCA. Portanto, o comportamento da redução por correlação foi, em geral, melhor em termos de concordância com as classes originais.

## d. Explique o comportamento dos índices com o k-means para as bases escolhidas

* **Correlação:** A redução por correlação parece preservar melhor a estrutura dos dados, resultando em partições iniciais mais compactas e bem definidas. Isso é refletido em índices DB menores e Silhouette scores mais altos para valores de
𝑘 menores.
* **PCA:** Embora o PCA capture a maior variação nos dados, ele pode não preservar as relações específicas que definem as classes. Isso resulta em índices DB mais altos e Silhouette scores mais baixos, especialmente à medida que o número de clusters aumenta.

O comportamento dos índices DB, Silhouette e ARI mostra que a redução por correlação é mais eficaz em manter a integridade das classes originais, resultando em partições mais compactas, bem separadas e com alta concordância com as classes verdadeiras. A PCA, enquanto útil, pode não ser tão eficaz em capturar a estrutura relevante para as classes, resultando em partições menos satisfatórias.